In [ ]:
import re
import json
import sympy as sp
import requests
from typing import List, Dict, Any
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline, PreTrainedModel, PreTrainedTokenizerFast
from torch.optim import Adam
import torch

In [ ]:
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

LLM_MODEL = "gemma-2-27b"  # Placeholder model
SEARCH_API_KEY = "YOUR_SEARCH_API_KEY"
MAX_HOTPOT_STEPS = 5
MAX_GSM8K_STEPS = 10

In [ ]:

# Function to get AI response
def get_openai_response(prompt, model="gpt-4o", log_file="./api_log_swirl.txt"):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}]
        )
        msg = response.choices[0].message.content.strip()

        if log_file != None:
            with open(log_file, "a") as f:
                f.write("PROMPT:\n\n" + prompt + "\n\n" + "RESPONSE\n\n" + msg + "\n\n")

        return msg
    except Exception as e:
        return f"Error: {e}"